In [1]:
import socket
import cv2
import numpy as np
from queue import Queue
from _thread import *
import pyrealsense2 as rs

In [2]:
queue_color = Queue()
queue_depth = Queue()

In [3]:
# client가 접속하면 color, depth image 정보를 string으로 encoding해서 client로 송신
def threaded(socket, addr, queue_color, queue_depth): 

    print('Connected by :', addr[0], ':', addr[1]) 

    while True: 

        try:
            data = socket.recv(1024)

            if not data: 
                print('Disconnected by ' + addr[0],':',addr[1])
                break

            stringData_color = queue_color.get()
            stringData_depth = queue_depth.get()
            stringData = stringData_color + stringData_depth
            socket.send(str(len(stringData_color)).ljust(16).encode())
            socket.send(str(len(stringData_depth)).ljust(16).encode())
            socket.send(stringData)

        except ConnectionResetError as e:

            print('Disconnected by ' + addr[0],':',addr[1])
            break
             
    socket.close() 

In [6]:
def camera_streaming(queue_color, queue_depth):
    
    ## 0~100에서 95의 이미지 품질로 설정 (default = 95)
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), 95]
    
    # Configure depth and color streams
    pipeline = rs.pipeline()
    config = rs.config()   

    # Set stream resolution
    #config.enable_stream(rs.stream.depth, DEPTHMAP_SIZE[1], DEPTHMAP_SIZE[0], rs.format.z16, 30)
    config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
    #config.enable_stream(rs.stream.depth, 1024, 768, rs.format.z16, 30)
    #config.enable_stream(rs.stream.color, IMAGE_SIZE[1], IMAGE_SIZE[0], rs.format.bgr8, 30)
    config.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)
    #config.enable_stream(rs.stream.color, 1920, 1080, rs.format.bgr8, 30)

    # Start streaming
    profile = pipeline.start(config)
    depth_sensor = profile.get_device().first_depth_sensor()
    depth_sensor.set_option(rs.option.visual_preset, 3)
    # Custom = 0, Default = 1, Hand = 2, HighAccuracy = 3, HighDensity = 4, MediumDensity = 5
    depth_scale = depth_sensor.get_depth_scale()
    align_to = rs.stream.color
    align = rs.align(align_to)

    try:
        while True:
            # Wait for a coherent pair of frames: depth and color
            frames = pipeline.wait_for_frames()
            aligned_frames = align.process(frames)

            #aligned_color_frame = aligned_frames.get_color_frame()
            #depth_frame = aligned_frames.get_depth_frame()

            aligned_depth_frame = aligned_frames.get_depth_frame()
            color_frame = aligned_frames.get_color_frame()
            #depth_frame = frames.get_depth_frame()
            #color_frame = frames.get_color_frame()
            if not aligned_depth_frame or not color_frame:
                continue

            #depth_to_color_extrins = depth_frame.profile.get_extrinsics_to(color_frame.profile)
            depth_intrins = aligned_depth_frame.profile.as_video_stream_profile().intrinsics
            #depth_intrins = depth_frame.profile.as_video_stream_profile().intrinsics
            #color_intrins = aligned_color_frame.profile.as_video_stream_profile().intrinsics
            #intrins = depth_intrins

            # Convert images to numpy arrays
            depth_image = np.asanyarray(aligned_depth_frame.get_data())
            color_image = np.asanyarray(color_frame.get_data())


            # 뭐 작업을 시작하라는 신호가 오면 client에서 쏘라고 하고, 그 뒤에 읽어들여야?
            #data = connection.recv(1024)

            # cv2. imencode(ext, img [, params])
            # encode_param의 형식으로 frame을 jpg로 이미지를 인코딩한다.
            result, color_img = cv2.imencode('.jpg', color_image, encode_param)
            result, depth_img = cv2.imencode('.png', depth_image, encode_param)

            # frame을 String 형태로 변환
            color_data = np.array(color_img)
            depth_data = np.array(depth_img)
            stringData_color = color_data.tostring()
            stringData_depth = depth_data.tostring()

            # color image와 depth image를 queue에 저장
            queue_color.put(stringData_color)
            queue_depth.put(stringData_depth)

    except Exception as e:
        print(e)

    finally:
        # Stop streaming
        pipeline.stop()

In [7]:
HOST='192.168.0.4'
PORT=8580
 
#TCP 사용
server_socket = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
print('Socket created')
 
#서버의 아이피와 포트번호 지정
server_socket.bind((HOST,PORT))
print('Socket bind complete')

# 클라이언트의 접속을 기다린다. (클라이언트 연결을 10개까지 받는다)
server_socket.listen(10)
print('Socket now listening')
 
# #연결, conn에는 소켓 객체, addr은 소켓에 바인드 된 주소
# connection, address = server_socket.accept()
# print('Server start')


start_new_thread(camera_streaming, (queue_color, queue_depth,))


while True: 

    print('wait')

    connection, addr = server_socket.accept()
    #connection2, addr2 = server_socket.accept()
    
    print('start thread')
    start_new_thread(threaded, (connection, addr, queue_color, queue_depth,))
    #start_new_thread(thread1, (connection1, addr1, queue_color,))
    #start_new_thread(thread2, (connection2, addr2, queue_depth,)) 

server_socket.close() 

Socket created
Socket bind complete
Socket now listening
wait
start thread
wait
('Connected by :', '192.168.0.4', ':', 43412)
('Disconnected by 192.168.0.4', ':', 43412)start thread
wait

('Connected by :', '192.168.0.4', ':', 43418)
('Disconnected by 192.168.0.4', ':', 43418)start thread
wait
 
('Connected by :', '192.168.0.4', ':', 43422)
('Disconnected by 192.168.0.4', ':', 43422)start thread
wait

('Connected by :', '192.168.0.4', ':', 43424)
('Disconnected by 192.168.0.4', ':', 43424)
start thread
wait('Connected by :', '192.168.0.4', ':', 43428)

('Disconnected by 192.168.0.4', ':', 43428)start thread

wait('Connected by :', '192.168.0.4', ':', 43430)

('Disconnected by 192.168.0.4', ':', 43430)
start thread
wait
('Connected by :', '192.168.0.4', ':', 43432)
('Disconnected by 192.168.0.4', ':', 43432)
start thread
wait('Connected by :', '192.168.0.4', ':', 43434)

('Disconnected by 192.168.0.4', ':', 43434)
Frame didn't arrive within 5000


KeyboardInterrupt: 

In [2]:
#socket에서 수신한 버퍼를 반환하는 함수
def recvall(sock, count):
    # 바이트 문자열
    buf = b''
    while count:
        newbuf = sock.recv(count)
        if not newbuf: return None
        buf += newbuf
        count -= len(newbuf)
    return buf
 
HOST='192.168.0.4'
PORT=8580
 
#TCP 사용
server_socket = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
print('Socket created')
 
#서버의 아이피와 포트번호 지정
server_socket.bind((HOST,PORT))
print('Socket bind complete')

# 클라이언트의 접속을 기다린다. (클라이언트 연결을 10개까지 받는다)
server_socket.listen(10)
print('Socket now listening')
 
#연결, conn에는 소켓 객체, addr은 소켓에 바인드 된 주소
connection, address = server_socket.accept()
print('Socket connected')

## 0~100에서 95의 이미지 품질로 설정 (default = 95)
encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), 95]

print('Start camera streaming')
# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()   

# Set stream resolution
#config.enable_stream(rs.stream.depth, DEPTHMAP_SIZE[1], DEPTHMAP_SIZE[0], rs.format.z16, 30)
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
#config.enable_stream(rs.stream.depth, 1024, 768, rs.format.z16, 30)
#config.enable_stream(rs.stream.color, IMAGE_SIZE[1], IMAGE_SIZE[0], rs.format.bgr8, 30)
config.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)
#config.enable_stream(rs.stream.color, 1920, 1080, rs.format.bgr8, 30)

# Start streaming
profile = pipeline.start(config)
depth_sensor = profile.get_device().first_depth_sensor()
depth_sensor.set_option(rs.option.visual_preset, 3)
# Custom = 0, Default = 1, Hand = 2, HighAccuracy = 3, HighDensity = 4, MediumDensity = 5
depth_scale = depth_sensor.get_depth_scale()
align_to = rs.stream.color
align = rs.align(align_to)

try:
    while True:
        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        aligned_frames = align.process(frames)

        #aligned_color_frame = aligned_frames.get_color_frame()
        #depth_frame = aligned_frames.get_depth_frame()

        aligned_depth_frame = aligned_frames.get_depth_frame()
        color_frame = aligned_frames.get_color_frame()
        #depth_frame = frames.get_depth_frame()
        #color_frame = frames.get_color_frame()
        if not aligned_depth_frame or not color_frame:
            continue

        #depth_to_color_extrins = depth_frame.profile.get_extrinsics_to(color_frame.profile)
        depth_intrins = aligned_depth_frame.profile.as_video_stream_profile().intrinsics
        #depth_intrins = depth_frame.profile.as_video_stream_profile().intrinsics
        #color_intrins = aligned_color_frame.profile.as_video_stream_profile().intrinsics
        #intrins = depth_intrins

        # Convert images to numpy arrays
        depth_image = np.asanyarray(aligned_depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())

        
        # 뭐 작업을 시작하라는 신호가 오면 client에서 쏘라고 하고, 그 뒤에 읽어들여야?
        #data = connection.recv(1024)
        
        print('Send data to client')
        if (True):
            # cv2. imencode(ext, img [, params])
            # encode_param의 형식으로 frame을 jpg로 이미지를 인코딩한다.
            result, color_img = cv2.imencode('.jpg', color_image, encode_param)
            result, depth_img = cv2.imencode('.png', depth_image, encode_param)

            # frame을 String 형태로 변환
            color_data = np.array(color_img)
            depth_data = np.array(depth_img)
            stringData_color = color_data.tostring()
            stringData_depth = depth_data.tostring()
            

            #서버에 데이터 전송
            #(str(len(stringData))).encode().ljust(16)
            server_socket.sendall((str(len(stringData_color))).encode().ljust(16) + stringData_color)
            print(str(len(stringData_color)))
            print(str(len(stringData_depth)))


except Exception as e:
    print(e)

finally:
    # Stop streaming
    pipeline.stop()




# while True:
#     # client에서 받은 stringData의 크기 (==(str(len(stringData))).encode().ljust(16))
    
#     length = recvall(connection, 16)
#     stringData1 = recvall(connection, int(length1))
#     data1 = np.fromstring(stringData1, dtype = 'uint8')
    
#     #data를 디코딩한다.
#     frame1 = cv2.imdecode(data1, cv2.IMREAD_COLOR)
#     cv2.imshow('ImageWindow',frame1)
#     key = cv2.waitKey(1)
    
#     if (key == 115):
#         # If 's' pressed, save color, depth images (far from table)
#         cv2.imwrite('/home/jhkim/Projects/rnb-planning/src/scripts/demo_202107/test_tcp/color.jpg', frame1) 
#         #cv2.imwrite('/home/jhkim/Projects/rnb-planning/src/scripts/demo_202107/test_tcp/depth.png', frame2)    
    
#     if (key == 27):
#         # If 'esc' pressed, stop streaming and exit
#         cv2.destroyAllWindows()
#         break

server_socket.close()

Socket created
Socket bind complete
Socket now listening
Socket connected
Start camera streaming
Send data to client
[Errno 32] Broken pipe


In [3]:
## 0~100에서 95의 이미지 품질로 설정 (default = 95)
encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), 95]

# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()   

# Set stream resolution
#config.enable_stream(rs.stream.depth, DEPTHMAP_SIZE[1], DEPTHMAP_SIZE[0], rs.format.z16, 30)
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
#config.enable_stream(rs.stream.depth, 1024, 768, rs.format.z16, 30)
#config.enable_stream(rs.stream.color, IMAGE_SIZE[1], IMAGE_SIZE[0], rs.format.bgr8, 30)
config.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)
#config.enable_stream(rs.stream.color, 1920, 1080, rs.format.bgr8, 30)

# Start streaming
profile = pipeline.start(config)
depth_sensor = profile.get_device().first_depth_sensor()
depth_sensor.set_option(rs.option.visual_preset, 3)
# Custom = 0, Default = 1, Hand = 2, HighAccuracy = 3, HighDensity = 4, MediumDensity = 5
depth_scale = depth_sensor.get_depth_scale()
align_to = rs.stream.color
align = rs.align(align_to)

try:
    while True:
        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        aligned_frames = align.process(frames)

        #aligned_color_frame = aligned_frames.get_color_frame()
        #depth_frame = aligned_frames.get_depth_frame()

        aligned_depth_frame = aligned_frames.get_depth_frame()
        color_frame = aligned_frames.get_color_frame()
        #depth_frame = frames.get_depth_frame()
        #color_frame = frames.get_color_frame()
        if not aligned_depth_frame or not color_frame:
            continue

        #depth_to_color_extrins = depth_frame.profile.get_extrinsics_to(color_frame.profile)
        depth_intrins = aligned_depth_frame.profile.as_video_stream_profile().intrinsics
        #depth_intrins = depth_frame.profile.as_video_stream_profile().intrinsics
        #color_intrins = aligned_color_frame.profile.as_video_stream_profile().intrinsics
        #intrins = depth_intrins
        #print(depth_intrins)
        #print(color_intrins)
        #print(depth_to_color_extrins)

        # Convert images to numpy arrays
        depth_image = np.asanyarray(aligned_depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())
        
        #print(depth_image.shape)
        #print(color_image.shape)
        #fusion_image = np.concatenate((color_image, depth_image), axis=1)
        
        # cv2. imencode(ext, img [, params])
        # encode_param의 형식으로 frame을 jpg로 이미지를 인코딩한다.
        #result1, color_img = cv2.imencode('.jpg', color_image, encode_param)
        #result2, depth_img = cv2.imencode('.png', depth_image, encode_param)
        result, img = cv2.imencode('.jpg', color_image, encode_param)
        
        # frame을 String 형태로 변환
        #data1 = np.array(color_img)
        #data2 = np.array(depth_img)
        data = np.array(img)
        #stringData1 = data1.tostring()
        #stringData2 = data2.tostring()
        stringData = data.tostring()

        #서버에 데이터 전송
        #(str(len(stringData))).encode().ljust(16)
        client_socket.sendall((str(len(stringData))).encode().ljust(16) + stringData)
        #client_socket.sendall((str(len(stringData2))).encode().ljust(16) + stringData2)
        
#         if (key == 27):
#             # If 'esc' pressed, stop streaming and exit
#             cv2.destroyAllWindows()
#             break

except Exception as e:
    print(e)

finally:
    # Stop streaming
    pipeline.stop()


name 'client_socket' is not defined
